In [41]:
import os
import json
from utils import read_tables, plot_all_columns
import pandas as pd
from sdv.utils import display_tables
from sdv.evaluation import evaluate
from sdmetrics.reports.multi_table import QualityReport
from sdmetrics.reports import utils
from sdv import Metadata

In [42]:
real_data = read_tables(
    '../data/rossman_store_sales/source', split_by="_", name_index=0)
synthetic_data = {}
synthetic_data['store'] = pd.read_csv('../data/rossman_store_sales/synthetic_realtabformer/realtabformer_store.csv', index_col=0)

/Users/martinjurkovic/Documents/MAG1-local/Project/Synthetic-data-generation-project/notebooks/utils.py:10: DtypeWarning:

Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.

/Users/martinjurkovic/Documents/MAG1-local/Project/Synthetic-data-generation-project/notebooks/utils.py:10: DtypeWarning:

Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.



In [43]:
synthetic_data['store']

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,839,c,a,240.0,10.0,2008.0,0,NaN,NaN,NaN
1,508,a,c,16270.0,12.0,2009.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,383,a,a,18640.0,NaN,NaN,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,510,a,c,8260.0,NaN,NaN,0,NaN,NaN,NaN
4,18,b,b,540.0,NaN,NaN,1,35.0,2010.0,"Mar,Jun,Sept,Dec"
...,...,...,...,...,...,...,...,...,...,...
1110,404,a,a,14560.0,8.0,2008.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
1111,642,d,c,2130.0,12.0,2007.0,0,NaN,NaN,NaN
1112,375,a,c,6970.0,2.0,2005.0,1,23.0,2015.0,"Mar,Jun,Sept,Dec"
1113,758,a,a,19780.0,NaN,NaN,1,45.0,2009.0,"Feb,May,Aug,Nov"


In [44]:
del real_data['train']
del real_data['test']
del real_data['sales']

In [45]:
# make all column names lowercase for all tables in real and synthetic data
for table_name, table in real_data.items():
    real_data[table_name].columns = [col.lower() for col in table.columns]
for table_name, table in synthetic_data.items():
    synthetic_data[table_name].columns = [col.lower() for col in table.columns]

# rename date column to sale_date for all tables in real and synthetic data
for table_name, table in real_data.items():
    real_data[table_name] = table.rename(columns={'date': 'sale_date'})
for table_name, table in synthetic_data.items():
    synthetic_data[table_name] = table.rename(columns={'date': 'sale_date'})

# add index column of 1 to n to sales table in real and synthetic data
for table_name, table in real_data.items():
    if table_name == 'sales':
        real_data[table_name]['index'] = range(1, len(table) + 1)

for table_name, table in synthetic_data.items():
    if table_name == 'sales':
        synthetic_data[table_name]['index'] = range(1, len(table) + 1)

In [46]:
metadata = Metadata()
metadata.add_table(
    name = 'store',
    data = real_data['store'],
    primary_key='store',
    # fields_metadata=store_fields_metadata,
)

In [47]:
metadata_name = '../models/rossman-gretelai-1.0-metadata.json'
metadata.to_json(metadata_name)

In [48]:
with open(metadata_name) as metadata_file:
    metadata = json.load(metadata_file)

In [49]:
evaluate(
    synthetic_data,
    real_data,
    aggregate=False,
    metadata=metadata)

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,KSComplement,Inverted Kolmogorov-Smirnov D statistic,0.936343,0.936343,0.0,1.0,MAXIMIZE,None
1,CSTest,Chi-Squared,0.999014,0.999014,0.0,1.0,MAXIMIZE,None


In [50]:
report = QualityReport()
report.generate(real_data, synthetic_data, metadata)

Creating report:   0%|          | 0/5 [00:00<?, ?it/s]/Users/martinjurkovic/.pyenv/versions/3.8.12/envs/ds_project/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.

/Users/martinjurkovic/.pyenv/versions/3.8.12/envs/ds_project/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.

Creating report: 100%|██████████| 5/5 [00:00<00:00, 23.84it/s]



Overall Quality Score: 93.94%

Properties:
Column Shapes: 94.75%
Column Pair Trends: 93.13%
Parent Child Relationships: NaN


/Users/martinjurkovic/.pyenv/versions/3.8.12/envs/ds_project/lib/python3.8/site-packages/sdmetrics/reports/multi_table/quality_report.py:130: RuntimeWarning:

Mean of empty slice



In [51]:
report.get_details(
    property_name='Column Shapes',
    table_name='store')

,Table,Column,Metric,Quality Score
0,store,competitiondistance,KSComplement,0.884163
1,store,competitionopensincemonth,KSComplement,0.953829
2,store,competitionopensinceyear,KSComplement,0.883416
3,store,promo2,KSComplement,0.954260
4,store,promo2sinceweek,KSComplement,0.972235
5,store,promo2sinceyear,KSComplement,0.970154
6,store,storetype,TVComplement,0.976682
7,store,assortment,TVComplement,0.964126
8,store,promointerval,TVComplement,0.968207


In [52]:
fig = report.get_visualization(
    property_name='Column Shapes',
    table_name='store')

fig.show()

In [53]:
fig = report.get_visualization(
    property_name='Column Pair Trends',
    table_name='store')

fig.show()

In [54]:
plot_all_columns(real_data, synthetic_data, metadata)

Could not plot store.store
sdtype of type 'id' not recognized.
